・mtgの場所（最大グループ数・一班の最大人数）・最上期・最小期・ファシリをする期・等を冒頭に宣言（できるだけマジックナンバーを作らない）


・他の人が勝手に生成してしまわないようにパスワードをかけても良い


・対面班をアルファベット、オンライン班をアラビア数字で割り振れば処理が楽になりそう


・リーダーは各班にいなくて良い？



・希望の出席形態と過去の席配置を読み込み


・ファシリ・議事録・司会を優先的に配置（議事録が新スタッフのみにならないように）


・残りを、期と前回の班がばらけるように配置（期で並び替えた後に前回の班・前々回の班で並び替えるとバラバラになる？）

・全員に通し番号を振って処理する

・スプシはプルダウンを使ってもらって入力を制限

・データフレームに条件を適用させてアクセスする方法が色々あるので統一させたい（loc,iat,condition等）

In [10]:
### sample code from Gemini

import pandas as pd

# サンプルデータフレーム作成
df = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ['A', 'B', 'C', 'D'], 'value': [10, 20, 30, 40]})
print("Before replacement:")
print(df)

# 参照テーブル（リスト形式）
reference_table = [0, 100, 200, 300, 400]  # インデックス1がid=1に対応

# 'name'列の条件に基づいて'id'列を上書き
def replace_id(row):
    if row['name'] == 'A':  # 条件：nameが'A'のとき
        if 0 < row['id'] < len(reference_table):  # idが参照テーブルの範囲内
            row['id'] = reference_table[row['id']]
    return row

# 'id'列を上書きする
df = df.apply(replace_id, axis=1)

print("\nAfter replacement:")
print(df)

Before replacement:
   id name  value
0   1    A     10
1   2    B     20
2   3    C     30
3   4    D     40

After replacement:
    id name  value
0  100    A     10
1    2    B     20
2    3    C     30
3    4    D     40


In [1]:
# 動作化環境を確認
import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

Python version
3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=12, micro=7, releaselevel='final', serial=0)


### 以下の環境で作成

Python version

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)
]
Version inf
o.
sys.version_info(major=3, minor=12, micro=7, releaselevel='final', serial=0)

In [2]:
# CSVデータをDL
import numpy as np
import pandas as pd
# df = pd.read_csv('source.csv')
df = pd.read_csv('for2024_10_30.csv')


# print(df)
print(df.head())
print(df.info())

    name  cohort attendanceFormat pre pre2 pre3  minuteTakerTimes  isLeader
0  17002      17               対面   3    C    F                 1         0
1  17003      17             Zoom   2    3    C                 1         0
2  17004      17               対面   D    B    A                 1         0
3  17005      17             Zoom   C    A    C                 1         0
4  17006      17             Zoom   1    2    1                 1         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              58 non-null     int64 
 1   cohort            58 non-null     int64 
 2   attendanceFormat  58 non-null     object
 3   pre               58 non-null     object
 4   pre2              57 non-null     object
 5   pre3              57 non-null     object
 6   minuteTakerTimes  58 non-null     int64 
 7   isLeader          58 no

In [3]:
# 最初に変数を設定

# 全体mtg実施場所
# 一方を使って他方はコメントアウト
                                            #←ここもユーザーからの入力で選べるようにした方が良いかも
Venue = 'a' # ANNEX
# Venue = 'j' # 情報統括センター

# ファシリテーションを担当する期
Faci=20

# 最上期と最下期
OldestCohort =17 # 最上期
YoungestCohort = 21 # 最下期

# 実施場所に応じて班員の最大値と、班数の最大値を設定→とりあえず簡単に実装するため n_groups でグループ数を指定する
# オンライン班については上限無しとする

n_groups = 8 # 作るグループの数

if(Venue == 'a'):
  # ANNEXの場合
  MNmembers = 8 # 一班あたりの最大人数：Maximum Numbers of members per group
  MNgroups = 6 # 班の数の最大：Maximum Numbers of groups
elif(Venue == 'j'):
  # 情報統括センターの場合
  MNmembers = 8
  MNgroups = 8

PhyGroups = ['A','B','C','D','E','F','G','H','I','J'] # 対面グループ参照用テーブル。対面参加：physically present

# 対面とオンラインのそれぞれの出席者をカウント　←　必要無いかも？
# T：対面　O：オンライン
# print(len(df.query('attendanceFormat == "対面" ')))
sumT=len(df.query('attendanceFormat == "対面" '))
sumO=len(df.query('attendanceFormat == "Zoom" '))
# print(sumT,sumO)
# 36 16

In [4]:
df

,name,cohort,attendanceFormat,pre,pre2,pre3,minuteTakerTimes,isLeader
0,17002,17,対面,3,C,F,1,0
1,17003,17,Zoom,2,3,C,1,0
2,17004,17,対面,D,B,A,1,0
3,17005,17,Zoom,C,A,C,1,0
4,17006,17,Zoom,1,2,1,1,0
5,17007,17,対面,0,F,B,1,0
6,17008,17,欠席,2,1,2,1,0
7,17009,17,対面,B,D,D,1,0
8,180010,18,Zoom,1,C,3,1,0
9,180011,18,対面,A,F,C,1,0


In [5]:
# 出席形態のリストを作成
AttendanceFormatList = ["対面","Zoom"]

# 所属している期のリストを作成。ファシリをする期優先的に配置するので先頭に移動させる。
CohortList = list(range(OldestCohort,YoungestCohort+1))
CohortList.insert(0,CohortList.pop(CohortList.index(Faci)))

In [6]:
cnt = 0
df['seat'] = -1 # 結果を格納用の列を追加

# ファシリの期の人をランダムにシャッフルして、班の数以内で通し番号を振る。番号を振った後でmodで最大班数以下にする。
for attendanceFormat in AttendanceFormatList:
    
    # とりあえずは、ここで作成するグループ数を指定
    if (attendanceFormat == "対面"):
        n_groups = 6
    elif (attendanceFormat == "Zoom"):
        n_groups = 4
    else:
        n_groups = -1
        
    for Cohort in CohortList:
        condition = (df['cohort'] == Cohort) & (df['attendanceFormat'] == attendanceFormat)
        # print(condition)
        df_filtered = df[condition]
        # print(df_filtered)
        random_indices = np.arange(cnt, cnt + len(df_filtered))  # 1から始まる連番を生成
        cnt = max(random_indices) % n_groups
        # print(random_indices)
        # print(cnt)
        # modを入れる
        np.random.shuffle(random_indices)  # ランダムにシャッフル
        df_filtered.index = random_indices
        
        # 元のデータフレームに反映
        df.loc[condition, 'seat'] = df_filtered.index
        df.loc[condition, 'seat'] = df.loc[condition, 'seat'] % n_groups
        # print(df)

    # if (attendanceFormat == "対面"):
        # todo テーブル参照

# df['seat'] = df['seat'] % n_groups
display(df)
# df.sort_values('seat')

,name,cohort,attendanceFormat,pre,pre2,pre3,minuteTakerTimes,isLeader,seat
0,17002,17,対面,3,C,F,1,0,4
1,17003,17,Zoom,2,3,C,1,0,0
2,17004,17,対面,D,B,A,1,0,2
3,17005,17,Zoom,C,A,C,1,0,3
4,17006,17,Zoom,1,2,1,1,0,2
5,17007,17,対面,0,F,B,1,0,5
6,17008,17,欠席,2,1,2,1,0,-1
7,17009,17,対面,B,D,D,1,0,3
8,180010,18,Zoom,1,C,3,1,0,2
9,180011,18,対面,A,F,C,1,0,0


In [ ]:
# 席配置用に調整

def adjustForSeating(row):    # 対面ならアルファベットに，Zoomなら数値＋1する
    if (row['attendanceFormat'] == "対面"):
        row['seat'] = PhyGroups[row['seat']]
    elif(row['attendanceFormat'] == "Zoom"):
        row['seat'] = row['seat'] + 1
    return row

df = df.apply(adjustForSeating, axis=1) # ここを複数回実行するとエラーが出る（アルファベットでテーブル参照をするため）
display(df)
# df.sort_values('seat')
df.to_csv('result.csv')

In [ ]:
display(df)
# df.sort_values('seat')
df.to_csv('result.csv')

In [ ]:
### backup
        # 元のデータフレームに反映
        df.loc[condition, 'seat'] = df_filtered.index
        # print(df)

    if (attendanceFormat == "対面"):
        # todo テーブル参照

df['seat'] = df['seat'] % n_groups
df.sort_values('seat')


### 途中
        # 元のデータフレームに反映
        df.loc[condition, 'seat'] = df_filtered.index
        # print(df)

    if (attendanceFormat == "対面"):
        # todo テーブル参照

df['seat'] = df['seat'] % n_groups
df.sort_values('seat')

In [27]:
# ファシリの期の人をランダムにシャッフルして、班の数以内で通し番号を振る。番号を振った後でmodで最大班数以下にする。
# df.loc[(df['cohort'] == Faci) & (df['attendanceFormat'] == "対面"), 'seat'] = range(1, len(df.loc[(df['cohort'] == Faci) & (df['attendanceFormat'] == "対面")]) + 1)
# df

# 継続は、ファシリ+1期から順に、期ごとにシャッフルして通し番号を振る。（前の期の番号をグローバル変数で管理しておくと偏りが出ないかも）

,name,cohort,attendanceFormat,pre,pre2,pre3,minuteTakerTimes,isLeader,seat
0,17002,17,対面,3,C,F,1,0,0
1,17003,17,対面,2,3,C,1,0,0
2,17004,17,対面,D,B,A,1,0,0
3,17005,17,対面,C,A,C,1,0,0
4,17006,17,Zoom,1,2,1,1,0,0
5,17007,17,対面,0,F,B,1,0,0
6,17008,17,Zoom,2,1,2,1,0,0
7,17009,17,対面,B,D,D,1,0,0
8,180010,18,Zoom,1,C,3,1,0,0
9,180011,18,対面,A,F,C,1,0,0


### 議事録担当の配置は後で実装

In [ ]:
# 議事録担当を優先的に配置
# print(df.query(('cohort == @Faci & attendanceFormat == "対面" ')))
df_T = df.query(('attendanceFormat == "対面" '))
min_MTTimes = df_T['minuteTakerTimes'].min()
num_min_MTTimes =len(df_T.query('minuteTakerTimes == @min_MTTimes '))
print(min_MTTimes,num_min_MTTimes)

# num_min_MTTimes が1人の場合
# num_min_MTTimes のcohortを重複無しで取り出す。個数が1で且つ youngestCohort と同じ場合は、 num_min_MTTimes をインクリメント
unq = df_T.query(('minuteTakerTimes == @min_MTTimes '))['cohort'].unique()
n_unq = len(unq)
if (n_unq == 1 & unq[0] == YoungestCohort ):

elif()
# num_min_MTTimes が全員 youngestCohort の場合

# 議事録担当者の議事録担当回数をインクリメント